In [ ]:
import requests
from time import sleep

In [39]:
%%time
headlines_dict = {}

url = "https://google-news.p.rapidapi.com/v1/search"

urls = []
headlines = []
summary = []
times = []
scrape_dates = []
websites = []
country = []

site = 'nytimes.com'

for i in range(1,29):
    #print(i)

    querystring = {"q":"women|woman|girl|girls|female|females|lady|ladies|she|her","lang":"en","sort_by":"relevancy","from":"2020-10-01","to":"2020-12-31","sources":site,"page":str(i),"media":"True"}

    headers = {
        'x-rapidapi-key': "3e81132ebdmshafd93b150db164ep13c18fjsn834550268d59",
        'x-rapidapi-host': "newscatcher.p.rapidapi.com"
        #'X-RapidAPI-Requests-Remaining'
        }

    sleep(randint(100,120))

    response = requests.request("GET", url, headers=headers, params=querystring).json()

    print(response)

    for j in range(len(response['articles'])):
        headlines.append(response['articles'][j]['title'])
        times.append(response['articles'][j]['published_date'])
        country.append(response['articles'][j]['country'])
        summary.append(response['articles'][j]['summary'])
        websites.append(response['articles'][j]['clean_url'])
        urls.append(response['articles'][j]['link'])

headlines_dict = {"url": urls, "headline": headlines,  "summary": summary, "time": times, "scrape_date": date.today().strftime("%d/%m/%Y"), "site": websites, "country": country}

headlines_df = pd.DataFrame.from_dict(headlines_dict)
number_of_articles = len(headlines_df)
# save results to csv
headlines_df.to_csv(f'../data/raw2/{number_of_articles}_{site}_articles.csv')
# print final statement
print("Saved data from " + str(number_of_articles) + " articles of "  + str(site) + " to .csv!")




{'message': 'You have exceeded the rate limit per hour for your plan, BASIC, by the API provider'}


KeyError: 'articles'

In [25]:
from datetime import date
date.today().strftime("%d/%m/%Y")

'13/05/2021'

In [20]:
for i in range(len(df2['articles'])):
    print(df2['articles'][i]['title'])

Kim Kardashian Has Learned Restraint (Published 2020)
Ben Affleck Tried to Drink Away the Pain. Now He's Trying Honesty.
Keke Palmer Needs Three Alarms to Wake Up (Published 2020)
This Millennial ‘Emma' Respects Its Elders (Published 2020)
Student Suspended Over Dreadlocks Is Invited to the Oscars (Published 2020)


In [21]:
title = response['articles'][i]['title']
published_date = response['articles'][i]['published_date']
author = response['author']
country = response['country']
urls = response['link']
websites = response['clean_url']

{'summary': "Continue reading the main storyDeAndre Arnold was told his dreadlocks violated his Texas high school's dress code. Now he has been invited to the Oscars by the celebrity couple Gabrielle Union and Dwyane Wade.Texas high school student DeAndre Arnold has said in interviews that he has been wearing his dreadlocks since seventh grade and has no plans to cut his hair.Credit...KHOU 11, via YouTubeBy Johnny Diaz and Jacey FortinPublished Feb. 1, 2020Updated Feb. 3, 2020A black student in Texas who was suspended because his high school said the way he wore his dreadlocks violated its dress code has been invited to the Oscars.",
 'country': 'US',
 'author': 'Johnny Diaz and Jacey Fortin',
 'link': 'https://www.nytimes.com/2020/02/01/us/DeAndre-Arnold-dreadlocks.html',
 'language': 'en',
 'media': 'https://static01.nyt.com/images/2020/02/01/multimedia/01xp-dreadlocks/01xp-dreadlocks-facebookJumbo.png?year=2020&h=549&w=1050&sig=0xfcedcb37cce8ea394d01ef0570256ec5',
 'title': 'Student

In [15]:
df2 = response.json()

In [17]:
df

{'status': 'ok',
 'total_hits': 20,
 'page': 1,
 'total_pages': 4,
 'page_size': 5,
 'articles': [{'summary': "Continue reading the main storyOne common cause is polycystic ovary syndrome, or PCOS, which is difficult to diagnose in adolescents.Period-tracking apps like Clue can help alert users to irregular cycles.Credit...CluePublished Feb. 24, 2020Updated Feb. 26, 2020I've written recently about teenagers having too-heavy periods and too-painful periods, and now I want to talk about too-irregular periods. These issues can overlap, but let's focus on what it can mean when an adolescent doesn't menstruate regularly — and on the question of how irregular is too irregular.",
   'country': 'US',
   'author': 'Perri Klass, M.D.',
   'link': 'https://www.nytimes.com/2020/02/24/well/family/teenagers-irregular-menstrual-periods-PCOS.html',
   'language': 'en',
   'media': 'https://static01.nyt.com/images/2020/02/24/well/24klass-PCOS/24klass-PCOS-facebookJumbo.jpg',
   'title': "When a Teenage

In [13]:
len(df['articles'])

5

In [32]:
len(df['articles'])

5

In [20]:
df.keys()

dict_keys(['status', 'total_hits', 'page', 'total_pages', 'page_size', 'articles', 'user_input'])